In [3]:
# Import necessary libraries
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import plotly.express as px
import plotly.io
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [4]:
# Load the dataset
df = pd.read_csv(r'C:\Users\brand\Downloads\Cleaned_Micro Center_Dataset (1).csv', parse_dates=['week'])
df.columns

Index(['week', 'year', 'quarter', 'month', 'week_number', 'sku',
       'functionality', 'functionality_code', 'color', 'color_code',
       'feat_main_page', 'feat_main_flag', 'vendor', 'weekly_sales', 'price',
       'revenue'],
      dtype='object')

In [5]:
# View data types of the columns
df.dtypes

week                  datetime64[ns]
year                           int64
quarter                        int64
month                          int64
week_number                    int64
sku                            int64
functionality                 object
functionality_code           float64
color                         object
color_code                     int64
feat_main_page                  bool
feat_main_flag                 int64
vendor                         int64
weekly_sales                   int64
price                        float64
revenue                      float64
dtype: object

In [6]:
# Week coverted to datetime and drop columns fucntionality and color.
df['week'] = pd.to_datetime(df['week'])
df.drop(columns=['functionality', 'color', 'feat_main_page'], inplace=True)


In [7]:
# Replace missing values in 'functionality_code' with the integer 10
df['functionality_code'] = df['functionality_code'].fillna(10).astype(int)

# Confirm the replacement
df['functionality_code'].isna().sum(), df['functionality_code'].dtype

# Check null and dtypes of the columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4400 entries, 0 to 4399
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   week                4400 non-null   datetime64[ns]
 1   year                4400 non-null   int64         
 2   quarter             4400 non-null   int64         
 3   month               4400 non-null   int64         
 4   week_number         4400 non-null   int64         
 5   sku                 4400 non-null   int64         
 6   functionality_code  4400 non-null   int64         
 7   color_code          4400 non-null   int64         
 8   feat_main_flag      4400 non-null   int64         
 9   vendor              4400 non-null   int64         
 10  weekly_sales        4400 non-null   int64         
 11  price               4400 non-null   float64       
 12  revenue             4400 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int64(10)
memo

In [8]:
# Convert feat_main_flag to a more readable format
df['Promotion'] = df['feat_main_flag'].map({0: 'No Promo', 1: 'Promo'})

In [9]:
# Plot box plot to compare weekly_sales with and without promotion
fig = px.box(
    df,
    x='Promotion',
    y='weekly_sales',
    title='Impact of Main Page Promotion on Weekly Sales',
    color='Promotion',
    labels={'weekly_sales': 'Weekly Sales Volume'}
)

fig.show()

"The distribution of weekly_sales is right-skewed for both groups, but the Promo group shows more high-end outliers. This suggests that main page promotions not only boost average sales but can also trigger occasional spikes in demand."

In [10]:
# Encoding binary variable with weekly_sales
avg_sales = df.groupby('Promotion')['weekly_sales'].mean().reset_index()


In [11]:
fig = px.bar(
    avg_sales,
    x='Promotion',
    y='weekly_sales',
    color='Promotion',
    title='Average Weekly Sales: Promo vs. No Promo',
    labels={'weekly_sales': 'Average Weekly Sales'}
    
)

fig.show()

"Chart shows the average weekly sales for both promoted and non-promoted products. Products featured on the main page had nearly twice the average weekly sales compared to those that weren’t promoted. This provides a clear indication that promotional placement has a positive impact on overall sales performance."

In [12]:
from scipy.stats import ttest_ind

# Separate the groups
promo_sales = df[df['feat_main_flag'] == 1]['weekly_sales']
no_promo_sales = df[df['feat_main_flag'] == 0]['weekly_sales']

# Run Welch’s t-test (doesn't assume equal variance)
t_stat, p_value = ttest_ind(promo_sales, no_promo_sales, equal_var=False)

print(f"T-Statistic: {t_stat:.2f}")
print(f"P-Value: {p_value:.4f}")

ModuleNotFoundError: No module named 'scipy'

“There is strong statistical evidence that weekly sales are significantly higher when a product is featured on the main page. The p-value of 0.0000 indicates this result is not due to chance — promotions are genuinely impacting sales.”

In [ ]:
import statsmodels.api as sm

# Regression model: weekly_sales ~ feat_main_flag
X = sm.add_constant(df['feat_main_flag'])  # Adds intercept
y = df['weekly_sales']

model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:           weekly_sales   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     47.11
Date:                Fri, 25 Apr 2025   Prob (F-statistic):           7.65e-12
Time:                        00:13:45   Log-Likelihood:                -31136.
No. Observations:                4400   AIC:                         6.228e+04
Df Residuals:                    4398   BIC:                         6.229e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const             60.9193      5.390     11.

“The regression model shows that being promoted on the main page is associated with an increase of ~60 weekly sales units on average, controlling for other factors. This reinforces the value of main-page exposure as a lever for sales growth.”

In [ ]:
# Total sales by vendor
vendor_perf = df.groupby('vendor')['weekly_sales'].sum().reset_index().sort_values(by='weekly_sales', ascending=False)

fig = px.bar(
    vendor_perf,
    x='vendor',
    y='weekly_sales',
    title='Total Weekly Sales by Vendor',
    labels={'weekly_sales': 'Total Sales'},
    color_continuous_scale='Blues',
    color='weekly_sales'
)
fig.show()

"Vendor 8 stands out as the top performer, generating the highest total weekly sales by a significant margin, followed by Vendors 6 and 5. This suggests that Vendor 8 consistently supplies high-demand products and may be a strong candidate for expanded partnerships or promotional focus. Further analysis of SKU-level performance within these vendors could help pinpoint the drivers of this success."

In [ ]:
sku_vendor = df.groupby(['vendor', 'sku'])['weekly_sales'].mean().reset_index()

fig = px.density_heatmap(
    sku_vendor,
    x='vendor',
    y='sku',
    z='weekly_sales',
    title='Average Weekly Sales: Vendor & SKU Performance',
    color_continuous_scale='Blues'
)
fig.show()

"Vendor 8 not only leads in total sales but also consistently supports high-performing SKUs—particularly SKU 30, which shows the strongest sales intensity overall. This indicates that Vendor 8 delivers both volume and reliability, making it a key contributor to revenue."

In [ ]:
# Group by functionality and calculate total revenue
functionality_revenue = df.groupby('functionality_code')['revenue'].sum().reset_index().sort_values(by='revenue', ascending=False)

fig = px.box(
    df,
    x='functionality_code',
    y='revenue',
    title='Revenue Distribution by Product Functionality',
    labels={'revenue': 'Revenue', 'functionality_code': 'Functionality Code'},
    points='all',
    color_discrete_sequence=['rgba(93, 164, 214, 0.5)']
)
fig.show()

"The box plot shows how much money each product makes within each functionality. Here, Functionality 11 stands out because its individual products tend to earn more, even if it doesn’t sell as many."

In [ ]:
fig = px.bar(
    functionality_revenue,
    x='functionality_code',
    y='revenue',
    title='Total Revenue by Product Functionality',
    labels={'revenue': 'Total Revenue', 'functionality_code': 'Functionality Code'},
    color='revenue',
    color_continuous_scale='Blues'
)
fig.show()

"Functionality 6 is winning by volume (lots of products selling steadily), while Functionality 11 is winning by product strength (fewer products, but stronger performance per item)."

In [ ]:
avg = df.groupby("week")[["weekly_sales","price"]].mean().reset_index()

# Create subplots: 2 rows, 1 column
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    subplot_titles=("Average Price Over Time", "Average Weekly Sales")
)

# Row 1: Average Price (now on top)
fig.add_trace(
    go.Scatter(x=avg["week"], y=avg["price"], mode='lines', name='Avg Price', line=dict(color='red')),
    row=1, col=1
)

# Row 2: Average Sales (now on bottom)
fig.add_trace(
    go.Scatter(x=avg["week"], y=avg["weekly_sales"], mode='lines', name='Avg Sales', line=dict(color='blue')),
    row=2, col=1
)

# Update layout
fig.update_layout(
    height=600,
    title_text="Average Price and Weekly Sales Over Time",
    showlegend=False
)

# Optional: rotate x-axis ticks
fig.update_xaxes(tickangle=30, row=2, col=1)

fig.show()

"While average prices steadily rise, weekly sales show an inverse pattern at key points. For example, during periods where price dips slightly, such as in late 2017 and summer 2018, sales surge — suggesting a negative price elasticity, where lower prices drive higher sales."

## 📢"This indicates that customers may be price-sensitive, and even small pricing adjustments could significantly impact weekly sales performance."

In [ ]:
monthly_sales = df.groupby('month')['weekly_sales'].sum().reset_index()

fig = px.bar(
    monthly_sales,
    x='month',
    y='weekly_sales',
    title='Total Sales by Month Across All Years',
    labels={'weekly_sales': 'Total Sales', 'month': 'Month'},
    text_auto='.2s',
    color='weekly_sales',
    color_continuous_scale='Blues'
)
fig.update_layout(xaxis=dict(tickmode='linear'))
fig.show()

“Sales peak mid-year, especially from May to September, making this the most profitable season. In contrast, February and Q4 show consistently lower sales, suggesting opportunities for targeted campaigns or off-season promotions to drive volume during slow periods.”

In [ ]:
weekly_avg = df.groupby('week_number')['weekly_sales'].mean().reset_index()

fig = px.line(
    weekly_avg,
    x='week_number',
    y='weekly_sales',
    title='Average Weekly Sales by Week Number',
    labels={'week_number': 'Week Number', 'weekly_sales': 'Avg Weekly Sales'}
)
fig.show()

“At Micro Center, peak sales periods consistently occur around late Q2 and early Q3, especially in June and September, likely driven by tech-seasonality like PC builds, back-to-school, or targeted promotions. Interestingly, Q4 does not exhibit the typical holiday surge seen in general retail, suggesting an opportunity to either ramp up campaigns during this time or focus budget earlier in the year.”

## Strategic Optimization Plan – Backed by Data

**💡Insight**: February is the weakest sales month (17k), followed by April and December.

Recommendation:

*   Use demand forecasting to reduce overstock risk during these periods. 
*   Focus on lean inventory strategies and use promo testing to   determine if these slumps can be lifted cost-effectively.

**💡Insight**: June and September are the highest grossing months (43k each), with clear spikes in weekly sales during weeks 24–26 and 37–38.

Recommendation:

*   Focus marketing and inventory efforts around late Q2 and early Q3. Launch seasonal bundles or flash sales timed with these periods to maximize ROI on high-traffic windows.



**💡Insight**: Visuals show an inverse relationship between price and sales — lower prices frequently align with sales spikes.

Recommendation:

*   Implement price elasticity modeling per SKU and test strategic markdowns during shoulder seasons (e.g., February, April, November) to smooth out low-demand dips without cutting into high-margin periods.